# PikaPikaGen: Training del Modello

Questo notebook automatizza il processo di setup e avvio del training per il modello PikaPikaGen.

I passaggi eseguiti sono:
1.  Clonazione del repository GitHub pubblico.
2.  Installazione delle dipendenze necessarie tramite `uv`.
3.  Esecuzione dello script di training `main.py`.

In [ ]:
print("Installazione delle dipendenze necessarie...")

# Assicurati che uv sia installato
%pip install uv
print("✅ uv installato con successo.")

# Controlla se torch è già installato
try:
    import torch
    print(f"✅ PyTorch già installato (versione: {torch.__version__})")
    torch_installed = True
except ImportError:
    print("❌ PyTorch non trovato, sarà installato")
    torch_installed = False

# Lista delle dipendenze principali del progetto
dependencies = [
    "transformers",
    "pandas",
    "tqdm",
    "matplotlib",
    "Pillow",
    "requests",
    "ipywidgets"
]

# Aggiungi torch e torchvision solo se non sono già installati
if not torch_installed:
    dependencies.extend(["torch", "torchvision"])

print("Installazione delle dipendenze con uv...")
deps_str = " ".join(dependencies)
if torch_installed:
    !uv pip install {deps_str}
else:
    !uv pip install {deps_str} --torch-backend=auto
print("✅ Dipendenze principali installate con successo.")


In [ ]:
import os

repo_url = "https://github.com/val-2/DeepLearning"
branch = "losses2"
repo_name = repo_url.split('/')[-1]

print(f"Clonazione del repository: {repo_url}")

# Check if we're already in the repo directory
current_dir = os.path.basename(os.getcwd())
if current_dir == repo_name:
    print(f"Già nella directory del repository '{repo_name}'. Aggiornamento...")
    !git fetch
    !git pull
    !git checkout {branch}
elif os.path.exists(repo_name):
    print(f"La directory '{repo_name}' esiste già. Aggiornamento del repository...")
    os.chdir(repo_name)
    !git fetch
    !git pull
    !git checkout {branch}
else:
    print(f"Clonazione del repository...")
    !git clone -b {branch} {repo_url}
    os.chdir(repo_name)

# Spostati nella directory del repository
print(f"Directory di lavoro corrente: {os.getcwd()}")

In [ ]:
print("Avvio dello script di training 'main.py'...")
%run pikapikagen/main.py

In [ ]:
import os
import glob
import re

def clean_checkpoints():
    """
    Elimina tutti i checkpoint tranne il migliore (best_model.pth.tar), l'ultimo
    e uno ogni 10 epoche
    """
    checkpoint_dir = "training_output/models"

    if not os.path.exists(checkpoint_dir):
        print(f"Directory {checkpoint_dir} non esiste.")
        return

    # Trova tutti i checkpoint con pattern checkpoint_epoch_*.pth.tar
    checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'checkpoint_epoch_*.pth.tar'))
    best_model_path = os.path.join(checkpoint_dir, 'best_model.pth.tar')

    if not checkpoint_files:
        print("Nessun checkpoint trovato.")
        return

    # Trova l'ultimo checkpoint basandosi sul numero di epoca
    try:
        latest_checkpoint = max(checkpoint_files, key=lambda f: int(re.search(r'epoch_(\d+)', f).group(1)))
        print(f"Ultimo checkpoint: {os.path.basename(latest_checkpoint)}")
    except (ValueError, AttributeError):
        print("Impossibile determinare l'ultimo checkpoint.")
        return

    # File da preservare
    files_to_keep = {latest_checkpoint}
    if os.path.exists(best_model_path):
        files_to_keep.add(best_model_path)
        print(f"Best model trovato: {os.path.basename(best_model_path)}")

    # Aggiungi checkpoint ogni 10 epoche
    for checkpoint_file in checkpoint_files:
        match = re.search(r'epoch_(\d+)', checkpoint_file)
        if match:
            epoch_num = int(match.group(1))
            if epoch_num % 10 == 0:  # Ogni 10 epoche
                files_to_keep.add(checkpoint_file)
                print(f"Conservato checkpoint ogni 10 epoche: {os.path.basename(checkpoint_file)}")

    # Elimina tutti gli altri checkpoint
    deleted_count = 0
    for checkpoint_file in checkpoint_files:
        if checkpoint_file not in files_to_keep:
            try:
                os.remove(checkpoint_file)
                print(f"Eliminato: {os.path.basename(checkpoint_file)}")
                deleted_count += 1
            except OSError as e:
                print(f"Errore nell'eliminazione di {checkpoint_file}: {e}")

    print(f"\n✅ Pulizia completata. Eliminati {deleted_count} checkpoint.")
    print(f"File conservati:")
    for kept_file in files_to_keep:
        if os.path.exists(kept_file):
            print(f"  - {os.path.basename(kept_file)}")

# Esegui la pulizia
clean_checkpoints()
